In [1]:
import qlat as q
import numpy as np

In [5]:
dst_pp = "/home/jhildebrand28/ktopipi/results/48I/auto-contract-I0D5-p/traj-1172/pipi_corr_psnk_psrc.lat"

ld_pp = q.load_lat_data(dst_pp)
ld_pp.info()[0][2]

['< 1 >  exprs[0]',
 '< sigma^dag(0) * sigma(-tsep) >  exprs[1]',
 '< wf_src(0) * sigma^dag(0) * pipi_i00(-tsep) > (ADT1_pps)  exprs[2]',
 '< wf_src(0) * sigma^dag(0) * pipi_i00(-tsep) > (ADT2_pps)  exprs[3]',
 '< wf_src(0) * sigma^dag(0) * pipi_i00(-tsep) > (ADT1_spp)  exprs[4]',
 '< wf_src(0) * sigma^dag(0) * pipi_i00(-tsep) > (ADT2_spp)  exprs[5]',
 '< wf_snk(0) * wf_src(0) * pipi_i00^dag(0) * pipi_i00(-tsep) > (ADT1)  exprs[6]',
 '< wf_snk(0) * wf_src(0) * pipi_i00^dag(0) * pipi_i00(-tsep) > (ADT2)  exprs[7]',
 '< wf_snk(0) * wf_src(0) * pipi_i00^dag(0) * pipi_i00(-tsep) > (ADT3)  exprs[8]',
 '< wf_snk(0) * wf_src(0) * pipi_i00^dag(0) * pipi_i00(-tsep) > (ADT4)  exprs[9]',
 '< wf_snk(1) * wf_src(0) * pipi_i00^dag(0) * pipi_i00(-tsep) > (ADT1)  exprs[10]',
 '< wf_snk(1) * wf_src(0) * pipi_i00^dag(0) * pipi_i00(-tsep) > (ADT2)  exprs[11]',
 '< wf_snk(1) * wf_src(0) * pipi_i00^dag(0) * pipi_i00(-tsep) > (ADT3)  exprs[12]',
 '< wf_snk(1) * wf_src(0) * pipi_i00^dag(0) * pipi_i00(-tsep) 

The goal here is to parse the expression strings returned in this info array for the lattide data such that for ANY two point correlation function we can use a hash table to lookup the expression index for a given two point (or three point?) correlator. This is incredibly useful for data analysis, because if we know the operators we used in the expressions, parsing the exact indices and patterns based on diagram type etc is very specific to a calculation, I want to run the lattice data through the parser and then be able to just look up the diagram types or expression indices from what I know I calculated. 

This requires us to parse each string, starting by breaking up each string into the individual terms. First, we will split the correlator expression, type, and index, and then within the correlator expression we will save the sink and source operator and wave function, along with the diagram type and expression index in a dictionary. From here, we have one dictionary for each expression, and I want to put the sink wave function and operator together to form an operator

In [ ]:
#parser class for lattice data loaded in from the qlattice auto-contractor
class LatData_parser:
    def __init__(self, lattice_data):
        self.lattice_data = lattice_data
        self.data_array = lattice_data.to_numpy()
        self.info_list = lattice_data.info()[0]
        self.nexp = info_list[1]
        self.expr_list = info_list[2]

    def parse_expr_names(self):
        expr_map = {}

        for i, expr_names in enumerate(self.expr_list):
            

In [53]:
class Expression:
    def __init__(self, expr_string):
        self.expr = expr_string

        self.op_src = None #these are operator objects that are present in the expression. This expression object will parse vevs, two point functions, and three point functions. 
        self.op_snk = None
        self.op_ins = None
        self.wf_src = 0
        self.wf_snk = 0
        self.idx = 0
        self.diagram_type = None
        
        pass

    #method takes in the raw string for each expression and saves each important piece as a class attribute
    def parse_expr(self):
        #the strings have three main components, an angle bracket section where the wave functions and operators are, a diagram type, and an expression index. 
        s = self.expr
        angle = re.compile(r'<(.*)>')
        diagram_type = re.compile(r'\((ADT[\d_]+)\)')
        index = re.compile(r'exprs\[(\d+)\]')
        
        corr_str = angle.search(s).group(1) #the 1 here ensures that the angle brackets are not captured. This string is then just the text in the angle bracket
        diagram_str = diagram_type.search(s).group(1)
        index_str = index.search(s).group(1) 

        self.idx = int(index_str)
        self.diagram_type = diagram_str

        #we must further break up the corr_str, by asterisks. this returns a list of each individual item. 
        corr_term_list = corr_str.split(" * ")
        term_dict_list = []
        
        for term in corr_term_list:
            term_dict_list.append(self.parse_term(term))   
            #print(self.parse_term(term))

        #we now have a list of dictionaries with all of the information we need to set the operators 
        self.assign_operators()

        
        
        #we need to deal with the fringe case of the counter expression, and the case of having 0 1 or 2 momentum projections

    #method that reads in a term from the correlator term list and returns a dictionary of the information it contains.
    #For wf terms we wish to save the fact that it is a wf term, its projection number, and if it is for the source or sink
    #for pipi terms we want to save the name and if it is a source or snk. 
    def parse_term(self,corr_term):
        #all momentum projection terms start with 'wf' 

        #capture the first bit of text in the string, then optionally capture the ^dag piece of the source operator term
        match = re.search(r'(\w+)(?:\^dag)?\(([^)]*)\)', corr_term)
        if not match:
            return None

        name = match.group(1) #this will be the name of the term no matter the type of term
        args = match.group(2) # this will give us the argument in the parenthesis, important for discerning the number of units of momentum. 

        if name.startswith('wf_'):
            return {
                'type': 'wavefunction',
                'role': name[3:],
                'p': int(args),
                'raw': corr_term,
            }
        else:
            return {
                'type': 'operator',
                'name': name,
                'is_dagger': ('^dag' in corr_term),
                'arg': args,
                'raw': corr_term,
            }
        
    
    #method that takes in the list of components from the angle bracket portion of the expression string and assigns operator objects based off of the information given. 
    def assign_operators(self, op_item_list):
        return 0

    
    def str_to_op(self):
        return 0
        
#struct like object that we will use t
class Operator: 
    def __init__(self):
        self.mom = 0
        self.name = None
        self.is_dagger = False

        pass

#this object acts as our main hashtable for all of the information we need.
class Expr_lookup:
    def __init__(self):
        self.corr_map
        self.vev_map

        pass

    #method that takes a single expression and adds its information to the relevant strucure
    def add_expr(self):
        return 0

    #mthod that returns a list of indices given a source and sink operator object. optional arguments allow for this to be used for vevs, two point functions, and three point functions. 
    def get_ind(op_src,op_ins=None,op_snk=None,expr_type=None):
        return 0


In [29]:
import re
s = '< wf_snk(2) * wf_src(0) * pipi_i00^dag(0) * pipi_i00(-tsep) > (ADT1)  exprs[14]'
s2 = '< wf_src(0) * sigma^dag(0) * pipi_i00(-tsep) > (ADT1_pps)  exprs[2]'
angle = re.compile(r'<(.*)>')
diagram_type = re.compile(r'\((ADT[\d_]+)\)')
index = re.compile(r'exprs\[(\d+)\]')
match = index.search(s2)
print(match.group(1))

2


In [13]:
corr_str

' wf_snk(2) * wf_src(0) * pipi_i00^dag(0) * pipi_i00(-tsep) '

In [31]:
l = angle.search(s2).group(1).split(" * ")
l

[' wf_src(0)', 'sigma^dag(0)', 'pipi_i00(-tsep) ']

In [54]:
s_test = '< wf_snk(2) * wf_src(0) * pipi_i00^dag(0) * pipi_i00(-tsep) > (ADT2)  exprs[7]'
test_expr = Expression(s_test)

In [55]:
test_expr.parse_expr()

{'type': 'wavefunction', 'role': 'snk', 'p': 2, 'raw': ' wf_snk(2)'}
{'type': 'wavefunction', 'role': 'src', 'p': 0, 'raw': 'wf_src(0)'}
{'type': 'operator', 'name': 'pipi_i00', 'is_dagger': True, 'arg': '0', 'raw': 'pipi_i00^dag(0)'}
{'type': 'operator', 'name': 'pipi_i00', 'is_dagger': False, 'arg': '-tsep', 'raw': 'pipi_i00(-tsep) '}
